In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

**Define training dataset**

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

batch_size = 32

trainset = torchvision.datasets.ImageFolder("../data/path-dt-msu-wsi/val", transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)


**Define model to be fine-tuned**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

In [ ]:
class VGG19(nn.Module):

    def __init__(self, pretrained=False):
        super().__init__()

        vgg19_pretrained = models.vgg19(pretrained=pretrained)
        self.backbone = vgg19_pretrained.features
        self.avgpool = vgg19_pretrained.avgpool
        
        self.fc1 = nn.Linear(in_features=25088, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=5)


    def forward(self, x):
        x = self.backbone(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        
        logits = self.fc3(x)

        return logits

In [ ]:
net = VGG19(pretrained=True)

**Define optimizer**

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

**Train**

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 0 and i > 0:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.5f}')
            running_loss = 0.0

print('Finished Training')

**Evaluation**

In [ ]:
from tqdm import tqdm

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(trainloader):
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network: {100 * correct // total} %')

In [ ]:
torch.save(net.state_dict(), "./model.pt")

In [ ]:
outputs.shape